In [2]:
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import contractions
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
plt.xticks(rotation=70)
pd.options.mode.chained_assignment = None
pd.set_option('display.max_colwidth', 100)
%matplotlib inline

In [3]:
df = pd.read_csv("C:/Users/Ankan SInha/Desktop/Final/NON-Irony/users/20baa1d4e5b84e345c0a3dbd002ff20c.csv")
df

,Tweets
0,"BPC, ITPF join hands to celebrate India National Tourism Day #URL# #URL#"
1,Pokemon Legends: Arceus Has a First Player Mode....Sort Of #URL# #URL#
2,"Farmers, opposition protest Spain's rural policies #URL# #URL#"
3,Washington trooper who defied state vaccine mandate and told gov to 'kiss my a--' dies from COVI...
4,"Shang-Chi Star Simu Liu Reveals He Contracted COVID-19 This Year, Encourages Vaccines #URL# #URL#"
5,Cops investigating assault of homeless man along Jalan Bukit Bintang #URL# #URL#
6,MSNBC deletes tweet slamming conservatives for favoring affirmative action when it benefits them...
7,"Chicago gun shop sees skyrocketing sales following lockdowns, crime spikes #URL# #URL#"
8,"Portuguese Socialists' lead for Jan. 30 election narrows, poll shows #URL# #URL#"
9,COVID strikes Kiribati: One of the last uninfected places on Earth #URL# #URL#


In [4]:
for col in df.columns:
    print(col, df[col].isnull().sum())

Tweets 0


In [5]:
rws = df.loc[:, ['Tweets']]

In [6]:
rws['Tweets_token'] = rws['Tweets'].apply(lambda x: [contractions.fix(word) for word in x.split()])
rws.head()

,Tweets,Tweets_token
0,"BPC, ITPF join hands to celebrate India National Tourism Day #URL# #URL#","[BPC,, ITPF, join, hands, to, celebrate, India, National, Tourism, Day, #URL#, #URL#]"
1,Pokemon Legends: Arceus Has a First Player Mode....Sort Of #URL# #URL#,"[Pokemon, Legends:, Arceus, Has, a, First, Player, Mode....Sort, Of, #URL#, #URL#]"
2,"Farmers, opposition protest Spain's rural policies #URL# #URL#","[Farmers,, opposition, protest, Spain's, rural, policies, #URL#, #URL#]"
3,Washington trooper who defied state vaccine mandate and told gov to 'kiss my a--' dies from COVI...,"[Washington, trooper, who, defied, state, vaccine, mandate, and, told, gov, to, 'kiss, my, a--',..."
4,"Shang-Chi Star Simu Liu Reveals He Contracted COVID-19 This Year, Encourages Vaccines #URL# #URL#","[Shang-Chi, Star, Simu, Liu, Reveals, He, Contracted, COVID-19, This, Year,, Encourages, Vaccine..."


In [7]:
rws['rating_description_str'] = [' '.join(map(str, l)) for l in rws['Tweets_token']]
rws['tokenized'] = rws['rating_description_str'].apply(word_tokenize)
rws['lower'] = rws['tokenized'].apply(lambda x: [word.lower() for word in x])
punc = string.punctuation
rws['no_punc'] = rws['lower'].apply(lambda x: [word for word in x if word not in punc])
stop_words = set(stopwords.words('english'))
new_stopwords = ["#", "url", "user",'a','an','the','of','or','is','it','for','have','has']
stopwrd = nltk.corpus.stopwords.words('english')
stopwrd.extend(new_stopwords)

rws['stopwords_removed'] = rws['no_punc'].apply(lambda x: [word for word in x if word not in new_stopwords])
rws['pos_tags'] = rws['stopwords_removed'].apply(nltk.tag.pos_tag)
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
rws['wordnet_pos'] = rws['pos_tags'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])
wnl = WordNetLemmatizer()
rws['lemmatized'] = rws['wordnet_pos'].apply(lambda x: [wnl.lemmatize(word, tag) for word, tag in x])
rws.head()

,Tweets,Tweets_token,rating_description_str,tokenized,lower,no_punc,stopwords_removed,pos_tags,wordnet_pos,lemmatized
0,"BPC, ITPF join hands to celebrate India National Tourism Day #URL# #URL#","[BPC,, ITPF, join, hands, to, celebrate, India, National, Tourism, Day, #URL#, #URL#]","BPC, ITPF join hands to celebrate India National Tourism Day #URL# #URL#","[BPC, ,, ITPF, join, hands, to, celebrate, India, National, Tourism, Day, #, URL, #, #, URL, #]","[bpc, ,, itpf, join, hands, to, celebrate, india, national, tourism, day, #, url, #, #, url, #]","[bpc, itpf, join, hands, to, celebrate, india, national, tourism, day, url, url]","[bpc, itpf, join, hands, to, celebrate, india, national, tourism, day]","[(bpc, NN), (itpf, NN), (join, NN), (hands, VBZ), (to, TO), (celebrate, VB), (india, JJ), (natio...","[(bpc, n), (itpf, n), (join, n), (hands, v), (to, n), (celebrate, v), (india, a), (national, a),...","[bpc, itpf, join, hand, to, celebrate, india, national, tourism, day]"
1,Pokemon Legends: Arceus Has a First Player Mode....Sort Of #URL# #URL#,"[Pokemon, Legends:, Arceus, Has, a, First, Player, Mode....Sort, Of, #URL#, #URL#]",Pokemon Legends: Arceus Has a First Player Mode....Sort Of #URL# #URL#,"[Pokemon, Legends, :, Arceus, Has, a, First, Player, Mode, ..., .Sort, Of, #, URL, #, #, URL, #]","[pokemon, legends, :, arceus, has, a, first, player, mode, ..., .sort, of, #, url, #, #, url, #]","[pokemon, legends, arceus, has, a, first, player, mode, ..., .sort, of, url, url]","[pokemon, legends, arceus, first, player, mode, ..., .sort]","[(pokemon, JJ), (legends, VBZ), (arceus, NN), (first, RB), (player, NN), (mode, NN), (..., :), (...","[(pokemon, a), (legends, v), (arceus, n), (first, r), (player, n), (mode, n), (..., n), (.sort, n)]","[pokemon, legends, arceus, first, player, mode, ..., .sort]"
2,"Farmers, opposition protest Spain's rural policies #URL# #URL#","[Farmers,, opposition, protest, Spain's, rural, policies, #URL#, #URL#]","Farmers, opposition protest Spain's rural policies #URL# #URL#","[Farmers, ,, opposition, protest, Spain, 's, rural, policies, #, URL, #, #, URL, #]","[farmers, ,, opposition, protest, spain, 's, rural, policies, #, url, #, #, url, #]","[farmers, opposition, protest, spain, 's, rural, policies, url, url]","[farmers, opposition, protest, spain, 's, rural, policies]","[(farmers, NNS), (opposition, NN), (protest, VBP), (spain, NN), ('s, POS), (rural, JJ), (policie...","[(farmers, n), (opposition, n), (protest, v), (spain, n), ('s, n), (rural, a), (policies, n)]","[farmer, opposition, protest, spain, 's, rural, policy]"
3,Washington trooper who defied state vaccine mandate and told gov to 'kiss my a--' dies from COVI...,"[Washington, trooper, who, defied, state, vaccine, mandate, and, told, gov, to, 'kiss, my, a--',...",Washington trooper who defied state vaccine mandate and told gov to 'kiss my a--' dies from COVI...,"[Washington, trooper, who, defied, state, vaccine, mandate, and, told, gov, to, 'kiss, my, a, --...","[washington, trooper, who, defied, state, vaccine, mandate, and, told, gov, to, 'kiss, my, a, --...","[washington, trooper, who, defied, state, vaccine, mandate, and, told, gov, to, 'kiss, my, a, --...","[washington, trooper, who, defied, state, vaccine, mandate, and, told, gov, to, 'kiss, my, --, d...","[(washington, NN), (trooper, NN), (who, WP), (defied, VBD), (state, NN), (vaccine, NN), (mandate...","[(washington, n), (trooper, n), (who, n), (defied, v), (state, n), (vaccine, n), (mandate, n), (...","[washington, trooper, who, defy, state, vaccine, mandate, and, tell, gov, to, 'kiss, my, --, dy,..."
4,"Shang-Chi Star Simu Liu Reveals He Contracted COVID-19 This Year, Encourages Vaccines #URL# #URL#","[Shang-Chi, Star, Simu, Liu, Reveals, He, Contracted, COVID-19, This, Year,, Encourages, Vaccine...","Shang-Chi Star Simu Liu Reveals He Contracted COVID-19 This Year, Encourages Vaccines #URL# #URL#","[Shang-Chi, Star, Simu, Liu, Reveals, He, Contracted, COVID-19, This, Year, ,, Encoura

In [8]:
rws.drop('tokenized', inplace=True, axis=1)

In [9]:
rws.head()

,Tweets,Tweets_token,rating_description_str,lower,no_punc,stopwords_removed,pos_tags,wordnet_pos,lemmatized
0,"BPC, ITPF join hands to celebrate India National Tourism Day #URL# #URL#","[BPC,, ITPF, join, hands, to, celebrate, India, National, Tourism, Day, #URL#, #URL#]","BPC, ITPF join hands to celebrate India National Tourism Day #URL# #URL#","[bpc, ,, itpf, join, hands, to, celebrate, india, national, tourism, day, #, url, #, #, url, #]","[bpc, itpf, join, hands, to, celebrate, india, national, tourism, day, url, url]","[bpc, itpf, join, hands, to, celebrate, india, national, tourism, day]","[(bpc, NN), (itpf, NN), (join, NN), (hands, VBZ), (to, TO), (celebrate, VB), (india, JJ), (natio...","[(bpc, n), (itpf, n), (join, n), (hands, v), (to, n), (celebrate, v), (india, a), (national, a),...","[bpc, itpf, join, hand, to, celebrate, india, national, tourism, day]"
1,Pokemon Legends: Arceus Has a First Player Mode....Sort Of #URL# #URL#,"[Pokemon, Legends:, Arceus, Has, a, First, Player, Mode....Sort, Of, #URL#, #URL#]",Pokemon Legends: Arceus Has a First Player Mode....Sort Of #URL# #URL#,"[pokemon, legends, :, arceus, has, a, first, player, mode, ..., .sort, of, #, url, #, #, url, #]","[pokemon, legends, arceus, has, a, first, player, mode, ..., .sort, of, url, url]","[pokemon, legends, arceus, first, player, mode, ..., .sort]","[(pokemon, JJ), (legends, VBZ), (arceus, NN), (first, RB), (player, NN), (mode, NN), (..., :), (...","[(pokemon, a), (legends, v), (arceus, n), (first, r), (player, n), (mode, n), (..., n), (.sort, n)]","[pokemon, legends, arceus, first, player, mode, ..., .sort]"
2,"Farmers, opposition protest Spain's rural policies #URL# #URL#","[Farmers,, opposition, protest, Spain's, rural, policies, #URL#, #URL#]","Farmers, opposition protest Spain's rural policies #URL# #URL#","[farmers, ,, opposition, protest, spain, 's, rural, policies, #, url, #, #, url, #]","[farmers, opposition, protest, spain, 's, rural, policies, url, url]","[farmers, opposition, protest, spain, 's, rural, policies]","[(farmers, NNS), (opposition, NN), (protest, VBP), (spain, NN), ('s, POS), (rural, JJ), (policie...","[(farmers, n), (opposition, n), (protest, v), (spain, n), ('s, n), (rural, a), (policies, n)]","[farmer, opposition, protest, spain, 's, rural, policy]"
3,Washington trooper who defied state vaccine mandate and told gov to 'kiss my a--' dies from COVI...,"[Washington, trooper, who, defied, state, vaccine, mandate, and, told, gov, to, 'kiss, my, a--',...",Washington trooper who defied state vaccine mandate and told gov to 'kiss my a--' dies from COVI...,"[washington, trooper, who, defied, state, vaccine, mandate, and, told, gov, to, 'kiss, my, a, --...","[washington, trooper, who, defied, state, vaccine, mandate, and, told, gov, to, 'kiss, my, a, --...","[washington, trooper, who, defied, state, vaccine, mandate, and, told, gov, to, 'kiss, my, --, d...","[(washington, NN), (trooper, NN), (who, WP), (defied, VBD), (state, NN), (vaccine, NN), (mandate...","[(washington, n), (trooper, n), (who, n), (defied, v), (state, n), (vaccine, n), (mandate, n), (...","[washington, trooper, who, defy, state, vaccine, mandate, and, tell, gov, to, 'kiss, my, --, dy,..."
4,"Shang-Chi Star Simu Liu Reveals He Contracted COVID-19 This Year, Encourages Vaccines #URL# #URL#","[Shang-Chi, Star, Simu, Liu, Reveals, He, Contracted, COVID-19, This, Year,, Encourages, Vaccine...","Shang-Chi Star Simu Liu Reveals He Contracted COVID-19 This Year, Encourages Vaccines #URL# #URL#","[shang-chi, star, simu, liu, reveals, he, contracted, covid-19, this, year, ,, encourages, vacci...","[shang-chi, star, simu, liu, reveals, he, contracted, covid-19, this, year, encourages, vaccines...","[shang-chi, star, simu, liu, reveals, he, contracted, covid-19, this, year, encourages, vaccines]","[(shang-chi, JJ), (star, NN), (simu, NN), (liu, NN), (reveals, NNS), (he, PRP), (contracted, VBD...","[(shang-chi, a), (star, n), (simu, n), (liu, n), (reveals, n), (he, n), (co

In [10]:
rws.to_csv('C:/Users/Ankan SInha/Desktop/Final/NON-Irony/user_processed/User_2.csv')

In [11]:
file = open('C:/Users/Ankan SInha/Desktop/Final/NON-Irony/user_processed/lem2.csv', encoding="utf8")
read_data = file.read()
d_per_word = read_data.split()
print('Total Words:', len(d_per_word))

Total Words: 2119


In [12]:
File = open('C:/Users/Ankan SInha/Desktop/Final/NON-Irony/user_processed/User_2.csv',encoding="utf8") #open file
lines = File.read() #read all lines
print (lines)
sentences = nltk.sent_tokenize(lines) #tokenize sentences
nouns = [] #empty to array to hold all nouns

for sentence in sentences:
     for word,pos in nltk.pos_tag(nltk.word_tokenize(str(sentence))):
         if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS'):
             nouns.append(word)
v=list(nouns)
v.remove("'")
''.join(v)

,Tweets,Tweets_token,rating_description_str,lower,no_punc,stopwords_removed,pos_tags,wordnet_pos,lemmatized
0,"BPC, ITPF join hands to celebrate India National Tourism Day #URL# #URL#","['BPC,', 'ITPF', 'join', 'hands', 'to', 'celebrate', 'India', 'National', 'Tourism', 'Day', '#URL#', '#URL#']","BPC, ITPF join hands to celebrate India National Tourism Day #URL# #URL#","['bpc', ',', 'itpf', 'join', 'hands', 'to', 'celebrate', 'india', 'national', 'tourism', 'day', '#', 'url', '#', '#', 'url', '#']","['bpc', 'itpf', 'join', 'hands', 'to', 'celebrate', 'india', 'national', 'tourism', 'day', 'url', 'url']","['bpc', 'itpf', 'join', 'hands', 'to', 'celebrate', 'india', 'national', 'tourism', 'day']","[('bpc', 'NN'), ('itpf', 'NN'), ('join', 'NN'), ('hands', 'VBZ'), ('to', 'TO'), ('celebrate', 'VB'), ('india', 'JJ'), ('national', 'JJ'), ('tourism', 'NN'), ('day', 'NN')]","[('bpc', 'n'), ('itpf', 'n'), ('join', 'n'), ('hands', 'v'), ('to', 'n'), ('celebrate', 'v'), ('india', 'a'), ('national'

ValueError: list.remove(x): x not in list

In [ ]:
File = open('C:/Users/Ankan SInha/Desktop/Final/NON-Irony/user_processed/lem2.csv', encoding="utf8") #open file
lines = File.read() #read all lines
sentences = nltk.sent_tokenize(lines) #tokenize sentences
nouns = [] #empty to array to hold all nouns

for sentence in sentences:
     for word,pos in nltk.pos_tag(nltk.word_tokenize(str(sentence))):
         if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS'):
             nouns.append(word)
nouns.sort()

In [18]:
nounx = list(dict.fromkeys(nouns))
#print(nounx)
x=len(nounx)
print(x)

2645


In [15]:
from nltk.corpus import wordnet as wn
def get_some_word_synonyms(word):
    word = word.lower()
    synonyms = []
    synsets = wordnet.synsets(word[1:])
    print(synsets)
   
for word in nounx:
    #if (synsets == 2):
    #print((word[1:]))
    synsets = get_some_word_synonyms(word)
    #print (synonyms)
    #x = len(synsets)
    #print (x)


[]
[]
[]
[]
[]
[Synset('personal_computer.n.01')]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[Synset('articulation.n.02'), Synset('union.n.08'), Synset('join.v.01'), Synset('join.v.02'), Synset('join.v.03'), Synset('join.v.04'), Synset('connect.v.03')]
[Synset('hands.n.01'), Synset('work_force.n.01'), Synset('hand.n.01'), Synset('hired_hand.n.01'), Synset('handwriting.n.01'), Synset('hand.n.04'), Synset('hand.n.05'), Synset('hand.n.06'), Synset('hand.n.07'), Synset('hand.n.08'), Synset('hand.n.09'), Synset('hand.n.10'), Synset('bridge_player.n.01'), Synset('hand.n.12'), Synset('hand.n.13'), Synset('hand.n.14'), Synset('pass.v.05'), Synset('hand.v.02')]
[]
[Synset('observe.v.06'), Synset('celebrate.v.02'), Synset('lionize.v.01')]
[Synset('india.n.01')]
[Synset('national.n.01'), Synset('national.a.01'), Synset('national.a.02'), Synset('national.a.03'), Synset('national.s.04'), Synset('home.s.03'), Synset('national.a.06'), Synset('national.a.07')]
[Synset('tourism.n.01')]
[Synset('day.n.01'), Synse

[Synset('rocket.n.04'), Synset('skyrocket.n.02'), Synset('rocket.v.01')]
[Synset('sale.n.01'), Synset('sale.n.02'), Synset('sale.n.03'), Synset('sale.n.04'), Synset('sale.n.05')]
[Synset('follow.v.01'), Synset('postdate.v.01'), Synset('follow.v.03'), Synset('follow.v.04'), Synset('comply.v.01'), Synset('follow.v.06'), Synset('follow.v.07'), Synset('follow.v.08'), Synset('adopt.v.01'), Synset('follow.v.10'), Synset('take_after.v.02'), Synset('trace.v.01'), Synset('watch.v.02'), Synset('succeed.v.02'), Synset('play_along.v.02'), Synset('keep_up.v.04'), Synset('come.v.05'), Synset('follow.v.18'), Synset('follow.v.19'), Synset('be.v.08'), Synset('surveil.v.01'), Synset('pursue.v.02'), Synset('follow.v.23'), Synset('stick_to.v.02')]
[]
[]
[]
[]
[Synset('arrow.n.01'), Synset('arrow.n.02')]
[]
[]
[Synset('january.n.01')]
[Synset('thirty.n.01'), Synset('thirty.s.01')]
[Synset('election.n.01'), Synset('election.n.02'), Synset('election.n.03'), Synset('election.n.04')]
[Synset('poll.n.01'), Syns

[Synset('up.v.01'), Synset('up.a.01'), Synset('astir.s.01'), Synset('improving.s.01'), Synset('up.s.04'), Synset('up.s.05'), Synset('up.s.06'), Synset('up.s.07'), Synset('up.s.08'), Synset('up.r.01'), Synset('up.r.02'), Synset('up.r.03'), Synset('up.r.04'), Synset('up.r.05')]
[Synset('sop.n.01'), Synset('sop.n.02'), Synset('standing_operating_procedure.n.01'), Synset('sop.v.01'), Synset('sop.v.02'), Synset('sop.v.03'), Synset('drench.v.04')]
[Synset('check.n.01'), Synset('assay.n.01'), Synset('check.n.03'), Synset('arrest.n.02'), Synset('confirmation.n.01'), Synset('check.n.06'), Synset('check_mark.n.01'), Synset('hindrance.n.01'), Synset('check.n.09'), Synset('check.n.10'), Synset('bridle.n.02'), Synset('check.n.12'), Synset('check.n.13'), Synset('check.v.01'), Synset('check.v.02'), Synset('see.v.10'), Synset('control.v.02'), Synset('check.v.05'), Synset('check.v.06'), Synset('check.v.07'), Synset('check.v.08'), Synset('match.v.01'), Synset('check.v.10'), Synset('discipline.v.01'), Sy

[Synset('girl.n.01'), Synset('female_child.n.01'), Synset('daughter.n.01'), Synset('girlfriend.n.02'), Synset('girl.n.05')]
[Synset('student.n.01'), Synset('scholar.n.01')]
[Synset('be.v.01'), Synset('be.v.02'), Synset('be.v.03'), Synset('exist.v.01'), Synset('be.v.05'), Synset('equal.v.01'), Synset('constitute.v.01'), Synset('be.v.08'), Synset('embody.v.02'), Synset('be.v.10'), Synset('be.v.11'), Synset('be.v.12'), Synset('cost.v.01')]
[Synset('police.n.01'), Synset('patrol.v.01')]
[Synset('defect.n.03'), Synset('flaw.n.02'), Synset('flaw.n.03'), Synset('flaw.v.01')]
[]
[Synset('anger.n.01'), Synset('wrath.n.02')]
[Synset('oman.n.01')]
[Synset('on.a.01'), Synset('on.a.02'), Synset('along.r.01'), Synset('on.r.02'), Synset('on.r.03')]
[]
[]
[Synset('fire.n.01'), Synset('fire.n.02'), Synset('fire.n.03'), Synset('fire.n.04'), Synset('fire.n.05'), Synset('ardor.n.03'), Synset('fire.n.07'), Synset('fire.n.08'), Synset('fire.n.09'), Synset('open_fire.v.01'), Synset('fire.v.02'), Synset('fire

[Synset('camp.n.01'), Synset('camp.n.02'), Synset('camp.n.03'), Synset('clique.n.01'), Synset('camp.n.05'), Synset('camp.n.06'), Synset('camp.n.07'), Synset('camp.n.08'), Synset('camp.v.01'), Synset('camp.v.02'), Synset('camp.v.03'), Synset('camp.s.01')]
[Synset('expert.n.01'), Synset('adept.s.01'), Synset('technical.s.04')]
[Synset('on.a.01'), Synset('on.a.02'), Synset('along.r.01'), Synset('on.r.02'), Synset('on.r.03')]
[Synset('clean_and_jerk.n.01'), Synset('clean.v.01'), Synset('clean.v.02'), Synset('houseclean.v.01'), Synset('cleanse.v.01'), Synset('clean.v.05'), Synset('clean.v.06'), Synset('clean.v.07'), Synset('clean.v.08'), Synset('scavenge.v.04'), Synset('clean.v.10'), Synset('clean.a.01'), Synset('clean.s.02'), Synset('clean.s.03'), Synset('clean.s.04'), Synset('clean.s.05'), Synset('clean.a.06'), Synset('clean.a.07'), Synset('clean.a.08'), Synset('uninfected.s.01'), Synset('clean.s.10'), Synset('clean.s.11'), Synset('blank.s.01'), Synset('clean.s.13'), Synset('clean.s.14'),

...

[Synset('certain.a.02'), Synset('certain.s.07'), Synset('certain.a.04'), Synset('sure.s.04'), Synset('certain.s.06'), Synset('sure.s.06'), Synset('sure.s.07'), Synset('sure.s.08'), Synset('indisputable.s.02'), Synset('surely.r.01')]
[Synset('constitutionalist.n.01')]
[Synset('court.n.01'), Synset('court.n.02'), Synset('court.n.03'), Synset('court.n.04'), Synset('court.n.05'), Synset('court.n.06'), Synset('motor_hotel.n.01'), Synset('court.n.08'), Synset('court.n.09'), Synset('court.n.10'), Synset('court.n.11'), Synset('woo.v.02'), Synset('woo.v.01'), Synset('court.v.03')]
[]
[]
[Synset('blackburn.n.01')]
[Synset('court.n.01'), Synset('court.n.02'), Synset('court.n.03'), Synset('court.n.04'), Synset('court.n.05'), Synset('court.n.06'), Synset('motor_hotel.n.01'), Synset('court.n.08'), Synset('court.n.09'), Synset('court.n.10'), Synset('court.n.11'), Synset('woo.v.02'), Synset('woo.v.01'), Synset('court.v.03')]
[]
[Synset('taliban.n.01')]
[]
[]
[]
[]
[]
[]
[Synset('google.n.01'), Synset(

[Synset('beef.n.01'), Synset('beef.n.02'), Synset('gripe.n.01'), Synset('gripe.v.01')]
[Synset('woman.n.01'), Synset('woman.n.02'), Synset('charwoman.n.01'), Synset('womanhood.n.02')]
[Synset('child.n.01'), Synset('child.n.02'), Synset('child.n.03'), Synset('child.n.04')]
[Synset('probe.n.01'), Synset('investigation.n.02')]
[Synset('division.n.01'), Synset('part.n.09'), Synset('division.n.03'), Synset('division.n.04'), Synset('division.n.05'), Synset('class.n.05'), Synset('division.n.07'), Synset('division.n.08'), Synset('division.n.09'), Synset('division.n.10'), Synset('division.n.11'), Synset('division.n.12')]
[Synset('budget.n.01'), Synset('budget.n.02'), Synset('budget.v.01')]
[]
[Synset('probe.n.01'), Synset('investigation.n.02')]
[]
[Synset('associate_in_nursing.n.01')]
[Synset('volt.n.01'), Synset('vanadium.n.01'), Synset('five.n.01'), Synset('v.n.04')]
[]
[]
[]
[Synset('connecticut.n.01'), Synset('computerized_tomography.n.01')]
[Synset('electron_volt.n.01')]
[Synset('prolifera

[Synset('large.a.01'), Synset('big.s.02'), Synset('bad.s.02'), Synset('big.s.04'), Synset('big.s.05'), Synset('big.s.06'), Synset('boastful.s.01'), Synset('big.s.08'), Synset('adult.s.01'), Synset('big.s.10'), Synset('big.s.11'), Synset('big.s.12'), Synset('big.s.13'), Synset('big.r.01'), Synset('boastfully.r.01'), Synset('big.r.03'), Synset('big.r.04')]
[Synset('battle.n.01'), Synset('struggle.n.01'), Synset('conflict.n.01'), Synset('battle.v.01')]
[Synset('dogfight.n.01'), Synset('hassle.n.02'), Synset('dogfight.n.03'), Synset('dogfight.n.04'), Synset('dogfight.v.01'), Synset('dogfight.v.02')]
[Synset('drone.n.01'), Synset('monotone.n.01'), Synset('dawdler.n.01'), Synset('drone.n.04'), Synset('drone.n.05'), Synset('drone.v.01'), Synset('drone.v.02')]
[Synset('army_intelligence.n.01'), Synset('artificial_intelligence.n.01'), Synset('three-toed_sloth.n.01'), Synset('artificial_insemination.n.01')]
[Synset('dogfight.n.01'), Synset('hassle.n.02'), Synset('dogfight.n.03'), Synset('dogfigh

[Synset('displace.v.01'), Synset('preempt.v.02'), Synset('displace.v.03'), Synset('move.v.02')]
[]
[Synset('ballad.n.01'), Synset('ballad.n.02'), Synset('put.v.01'), Synset('lay.v.02'), Synset('lay.v.03'), Synset('lay.v.04'), Synset('lay.v.05'), Synset('lie.v.01'), Synset('lie.v.02'), Synset('dwell.v.02'), Synset('lie.v.04'), Synset('lie.v.05'), Synset('lie.v.06'), Synset('lie_down.v.01'), Synset('laic.s.01'), Synset('lay.s.02')]
[]
[]
[]
[Synset('download.v.01')]
[Synset('trojan.n.01'), Synset('trojan.n.02'), Synset('trojan.a.01')]
[]
[]
[]
[]
[Synset('hem.n.01'), Synset('hem.n.02'), Synset('hem.v.01'), Synset('hem.v.02')]
[Synset('trip.n.01'), Synset('trip.n.02'), Synset('slip.n.07'), Synset('trip.n.04'), Synset('tripper.n.04'), Synset('trip.n.06'), Synset('trip.n.07'), Synset('stumble.v.02'), Synset('trip.v.02'), Synset('travel.v.03'), Synset('trip.v.04'), Synset('trip.v.05')]
[Synset('roll.v.12'), Synset('reeve.v.01'), Synset('reeve.v.02'), Synset('reeve.v.03')]
[]
[]
[Synset('alle

[Synset('due.n.01'), Synset('due.n.02')]
[Synset('bankrupt.n.01'), Synset('insolvent.a.01')]
[]
[Synset('due.n.01'), Synset('due.n.02'), Synset('due.a.01'), Synset('due.s.02'), Synset('due.a.03'), Synset('ascribable.s.01'), Synset('due.r.01')]
[Synset('declare.v.01'), Synset('announce.v.02'), Synset('declare.v.03'), Synset('declare.v.04'), Synset('declare.v.05'), Synset('declare.v.06'), Synset('declare.v.07'), Synset('declare.v.08')]


In [19]:
Average_data= ((1502/2645)*100)
print (Average_data)

56.78638941398866
